In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.getcwd()))

In [2]:
from hftbacktest import HftBacktest, FeedLatency, Linear, SquareProbQueueModel
import numpy as np
import pandas as pd

In [3]:
def convert_to_orderbook(hbt):
    local_time = []
    bid = []
    ask = []
    bid_volume = []
    ask_volume = []
    
    prev_timestamp = 0
    timestamp_val = 0
    
    while hbt.run:
        if not hbt.elapse(100_000):
            dict_order_book = {'Local time': local_time, 'Ask': ask, 'Bid': bid, 'AskVolume': ask_volume, 'BidVolume': bid_volume }
            df = pd.DataFrame(dict_order_book)
            return False, df
        
        if (hbt.timestamp != prev_timestamp):
            prev_timestamp = hbt.timestamp
            timestamp_val = hbt.timestamp
        else:
            timestamp_val += 1
        local_time.append(timestamp_val)
        
        for tick_price in range(hbt.best_ask_tick, hbt.high_ask_tick + 1):
            if tick_price in hbt.ask_depth:
                ask_volume.append(hbt.ask_depth[tick_price])
                ask.append(round(tick_price * hbt.tick_size, 3))
                break

        for tick_price in range(hbt.best_bid_tick, hbt.low_bid_tick - 1, -1):
            if tick_price in hbt.bid_depth:
                bid_volume.append(hbt.bid_depth[tick_price])
                bid.append(round(tick_price * hbt.tick_size, 3))
                break

    dict_order_book = {'Local time': local_time, 'Ask': ask, 'Bid': bid, 'AskVolume': ask_volume, 'BidVolume': bid_volume }
    df = pd.DataFrame(dict_order_book)
    return True, df

In [4]:
file_name = 'btcusdt_20230613'
eod_file_name = 'btcusdt_20230612'

btcusdt = np.load(f'./data/{file_name}.npz')['data']
btcusdt_eod = np.load(f'./data/{eod_file_name}_eod.npz')['data']

hbt = HftBacktest(
    btcusdt,
    tick_size=0.1,
    lot_size=0.001,
    maker_fee=-0.00005,
    taker_fee=0.0007,
    order_latency=FeedLatency(),
    asset_type=Linear,
    snapshot=btcusdt_eod
)

In [5]:
res, order_book = convert_to_orderbook(hbt)
order_book

,Local time,Ask,Bid,AskVolume,BidVolume
0,1686589200120958,25814.1,25814.0,1.389,12.963
1,1686589200227102,25814.1,25814.0,1.524,12.969
2,1686589200320551,25814.1,25814.0,1.524,12.950
3,1686589200424194,25814.1,25814.0,1.524,12.930
4,1686589200535408,25814.1,25814.0,1.514,12.928
...,...,...,...,...,...
863994,1686675599531074,25850.9,25850.8,5.142,6.126
863995,1686675599623973,25850.9,25850.8,5.142,6.150
863996,1686675599732314,25850.9,25850.8,4.977,6.161
863997,1686675599824283,25850.9,25850.8,4.977,6.161


In [6]:
order_book.to_csv(os.path.abspath(os.path.join(os.getcwd(), f'../deep_learning/simulator/utils/data/{file_name}.csv')), index=False)